In [41]:
!pip install pandas
import sys
!{sys.executable} -m pip install jupyter_dash
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [160]:
spacex_df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_value = spacex_df['Payload Mass (kg)'].max()
min_value = spacex_df['Payload Mass (kg)'].min()
#spacex_df['VersionE']=spacex_df['Booster Version'].str.split().str[1]
spacex_df.head()
listV=[]
for i in range(0,11000,1000):
    listV.append(i)
SlideMarks=dict(zip(listV,listV))

print(SlideMarks)

{0: 0, 1000: 1000, 2000: 2000, 3000: 3000, 4000: 4000, 5000: 5000, 6000: 6000, 7000: 7000, 8000: 8000, 9000: 9000, 10000: 10000}


In [167]:

app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                html.Div(dcc.Dropdown(id='site-dropdown',
                                  options=[{'label': 'All Sites', 'value': 'ALL'},
                                           {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                           {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                           {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                           {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                    value='ALL',
                                    placeholder="place holder here",
                                    searchable=True
                                )
                                ),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                         min=0, max=10000, step=1000,
                                         marks=SlideMarks,
                                         value=[min_value, max_value])),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [168]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
''''''
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):    
    filtered_df = spacex_df
    if entered_site == 'ALL':        
        filtered_df=filtered_df[filtered_df['class']==1]
        LaunchSites=filtered_df.groupby(['Launch Site']).groups.keys()
        Successfultimes=filtered_df.groupby(['Launch Site']).size().tolist()
        fig = px.pie(values=Successfultimes, 
        names=LaunchSites, 
        title='20231031')
        return fig
    else:
        filtered_df=filtered_df[filtered_df['Launch Site']==entered_site]
        TC=len(filtered_df[filtered_df['class']==1])
        FC=len(filtered_df[filtered_df['class']==0])      
        fig = px.pie(values=[TC,FC], 
        names=['Successful','Fail'], 
        title='20231031-1')
        return fig
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
             
def get_scatter(entered_site,enter_slideValue):    
    minV=enter_slideValue[0]
    maxV=enter_slideValue[1]
    df=spacex_df[spacex_df['Payload Mass (kg)'].between(minV,maxV)]
#    return px.scatter(df,x='Payload Mass (kg)',y='class',color='Booster Version Category')      
    if entered_site == 'ALL':                
        fig=px.scatter(df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='ALL',render_mode='webgl')
        return fig
    else:
        SpecificSite_df=df[df['Launch Site']==entered_site]
        fig=px.scatter(SpecificSite_df,x='Payload Mass (kg)',y='class',color='Booster Version Category', title=entered_site,render_mode='webgl')
        return fig
''''''
# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline')